In [1]:
from jupyter_dash import JupyterDash  # <-- This is the key change
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import pandas as pd
from IPython.display import display, HTML

# Load and prepare the data
try:
    # Load the CSV file
    df = pd.read_csv('aac_shelter_outcomes.csv')
    
    # Data cleaning - first check if we have location data
    location_cols = ['latitude', 'longitude']
    if all(col in df.columns for col in location_cols):
        df = df.dropna(subset=location_cols)  # Remove rows without coordinates
    else:
        print(f"Warning: Required columns {location_cols} not found - trying to find alternatives")
        # Try to find any columns that might contain coordinates
        possible_lat = [col for col in df.columns if 'lat' in col.lower()]
        possible_lon = [col for col in df.columns if 'lon' in col.lower() or 'lng' in col.lower()]
        
        if possible_lat and possible_lon:
            df = df.rename(columns={
                possible_lat[0]: 'latitude',
                possible_lon[0]: 'longitude'
            })
            df = df.dropna(subset=['latitude', 'longitude'])
            print(f"Using columns '{possible_lat[0]}' and '{possible_lon[0]}' as location data")
        else:
            print("No suitable location columns found - map will not work")
            # Add dummy coordinates if none found
            df['latitude'] = 30.75
            df['longitude'] = -97.48
    
    # Reset index after cleaning
    df = df.reset_index(drop=True)
    
    # Convert numeric columns safely
    numeric_cols = ['age_upon_outcome_in_weeks', 'latitude', 'longitude']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print(f"Successfully loaded {len(df)} records")
    print("Available columns:", df.columns.tolist())
    
except Exception as e:
    print(f"Error loading data: {str(e)}")
    # Create empty dataframe with expected columns
    df = pd.DataFrame(columns=['name', 'animal_type', 'breed', 'age_upon_outcome_in_weeks', 
                             'outcome_type', 'latitude', 'longitude'])

app = JupyterDash(__name__)

# Create the layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Animal Shelter Outcomes Dashboard'))),
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": col, "id": col} for col in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left'},
        row_selectable='single',
        selected_rows=[] if df.empty else [0],
        page_size=10,
        filter_action='native'
    ),
    html.Br(),
    html.Hr(),
    html.Div(id='map-id')
])

# Callback to update the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or not index:
        return html.Div("Select a row to view location")
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    
    # Get data with defaults
    lat = dff.iloc[row].get('latitude', 30.75)  # Default to Austin, TX
    lon = dff.iloc[row].get('longitude', -97.48)
    name = dff.iloc[row].get('name', 'Unknown')
    animal_type = dff.iloc[row].get('animal_type', 'Unknown')
    
    # Try to convert to float
    try:
        lat = float(lat)
        lon = float(lon)
    except (ValueError, TypeError):
        lat, lon = 30.75, -97.48  # Default coordinates
    
    return dl.Map(
        style={'width': '100%', 'height': '500px'},
        center=[lat, lon], 
        zoom=12,
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(f"{animal_type}: {name}"),
                    dl.Popup([
                        html.H3("Animal Details"),
                        html.P(f"Name: {name}"),
                        html.P(f"Type: {animal_type}"),
                        html.P(f"Breed: {dff.iloc[row].get('breed', 'N/A')}"),
                        html.P(f"Age: {dff.iloc[row].get('age_upon_outcome_in_weeks', 'N/A')} weeks"),
                        html.P(f"Outcome: {dff.iloc[row].get('outcome_type', 'N/A')}")
                    ])
                ]
            )
        ]
    )

app.run_server(mode='inline', port=8050)

Using columns 'location_lat' and 'location_long' as location data
Successfully loaded 10000 records
Available columns: ['Unnamed: 0', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'latitude', 'longitude', 'age_upon_outcome_in_weeks']
